In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Mô hình Random Forrest
from sklearn.ensemble import RandomForestClassifier
#SMOTE
from imblearn.over_sampling import SMOTE
#Mô hình XG Boost
import xgboost as xgb

#MLP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, auc
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [3]:
data=pd.read_csv("/content/drive/MyDrive/Datasets/Give Me Some Credit Cards/GiveMeSomeCredit-training.csv")

**CHECKING**

In [4]:
data=data.drop(columns=['Unnamed: 0'])
data

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [6]:
#Kiểm tra xem có dòng nào Null không
data.isnull().sum()

,0
SeriousDlqin2yrs,0
RevolvingUtilizationOfUnsecuredLines,0
age,0
NumberOfTime30-59DaysPastDueNotWorse,0
DebtRatio,0
MonthlyIncome,29731
NumberOfOpenCreditLinesAndLoans,0
NumberOfTimes90DaysLate,0
NumberRealEstateLoansOrLines,0
NumberOfTime60-89DaysPastDueNotWorse,0


In [7]:
#Số lượng null không nhiều, ta có thể drop bớt
data= data.dropna()

In [9]:
#Dễ thấy, thông qua Corr-matrix, ta thấy biến mục tiêu không có sự tuyến tính với bất kì biến mục tiêu nào
#Vì vậy, có thể phân tích dữ liệu bằng các mô hình Classification phi tuyến
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
SeriousDlqin2yrs,1.000000,-0.002386,-0.102685,0.123159,-0.003291,-0.019746,-0.027445,0.111149,-0.003005,0.093836,0.046704
RevolvingUtilizationOfUnsecuredLines,-0.002386,1.000000,-0.005476,-0.001194,-0.000243,0.007124,-0.010880,-0.000931,0.005895,-0.000955,0.002347
age,-0.102685,-0.005476,1.000000,-0.049177,-0.001260,0.037717,0.187112,-0.047889,0.065110,-0.043501,-0.206397
NumberOfTime30-59DaysPastDueNotWorse,0.123159,-0.001194,-0.049177,1.000000,-0.001981,-0.010217,-0.043868,0.975259,-0.024036,0.980131,0.003538
DebtRatio,-0.003291,-0.000243,-0.001260,-0.001981,1.000000,-0.028712,0.008685,-0.002703,0.019574,-0.001801,0.009803
MonthlyIncome,-0.019746,0.007124,0.037717,-0.010217,-0.028712,1.000000,0.091455,-0.012743,0.124959,-0.011116,0.062647
NumberOfOpenCreditLinesAndLoans,-0.027445,-0.010880,0.187112,-0.043868,0.008685,0.091455,1.000000,-0.072249,0.426536,-0.061505,0.038342
NumberOfTimes90DaysLate,0.111149,-0.000931,-0.047889,0.975259,-0.002703,-0.012743,-0.072249,1.000000,-0.040573,0.989277,-0.003779
NumberRealEstateLoansOrLines,-0.003005,0.005895,0.065110,-0.024036,0.019574,0.124959,0.426536,-0.040573,1.000000,-0.034118,0.118997
NumberOfTime60-89DaysPastDueNotWorse,0.093836,-0.000955,-0.043501,0.980131,-0.001801,-0.011116,-0.061505,0.989277,-0.034118,1.000000,-0.005092


In [13]:
#Qua ma trận corr, ta thấy có một số cặp columns có quan hệ tuyến tính với nhau, vì vậy ta sẽ drop bớt 1 trong 2 ở mỗi cặp
data=data.drop(columns=['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans'])

In [14]:
for i in data.columns:
  print(data[i].value_counts())
  print('\n')

SeriousDlqin2yrs
0    111912
1      8357
Name: count, dtype: int64


RevolvingUtilizationOfUnsecuredLines
0.000000    7871
1.000000    7461
1.000000      15
0.950100       8
0.954092       6
            ... 
0.307818       1
0.078050       1
0.821609       1
0.224809       1
0.850283       1
Name: count, Length: 103063, dtype: int64


age
49     3217
48     3163
47     3134
46     3117
50     3105
       ... 
99        3
103       2
102       2
101       2
0         1
Name: count, Length: 83, dtype: int64


DebtRatio
0.000000    2491
1.000000      24
8.000000      16
0.333333      15
0.001200      15
            ... 
0.318144       1
0.401104       1
0.307895       1
0.219356       1
0.249908       1
Name: count, Length: 109658, dtype: int64


MonthlyIncome
5000.0     2757
4000.0     2106
6000.0     1934
3000.0     1758
0.0        1634
           ... 
16084.0       1
10523.0       1
55250.0       1
9242.0        1
7647.0        1
Name: count, Length: 13594, dtype: int64


NumberOfTimes

**PREDICTION**

In [18]:
#Hàm đánh giá hiệu suất
def print_scores(y_test, y_pred):
  print(classification_report(y_test, y_pred))

In [19]:
#Hàm chia tập train-test và MinMaxScaler
def preprocess_data(data, test_size=0.2, random_state=42):

    # Chia tập train-test
    X = data.drop(columns=['SeriousDlqin2yrs'], axis=1)
    y = data['SeriousDlqin2yrs']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # MinMaxScaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

**1.Random Forrest**

In [20]:
#Random Forrest
X_train, X_test, y_train, y_test = preprocess_data(data)

model_RF= RandomForestClassifier(n_estimators=200, random_state=42)
model_RF.fit(X_train, y_train)

y_pred = model_RF.predict(X_test)

print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.99      0.96     22356
           1       0.57      0.15      0.23      1698

    accuracy                           0.93     24054
   macro avg       0.75      0.57      0.60     24054
weighted avg       0.91      0.93      0.91     24054



**2.XGBOOST**

In [23]:
#XGBOOST
X_train, X_test, y_train, y_test = preprocess_data(data)

model_XGB = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_XGB.fit(X_train, y_train)

y_pred = model_XGB.predict(X_test)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.99      0.96     22356
           1       0.51      0.15      0.23      1698

    accuracy                           0.93     24054
   macro avg       0.72      0.57      0.60     24054
weighted avg       0.91      0.93      0.91     24054



**3.MLP Classifier**

In [32]:
X_train, X_test, y_train, y_test = preprocess_data(data)
y_train = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)  # Lưu giữ y_test gốc để sau đó chuyển lại

model = Sequential()
model.add(Dense(10, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=10, validation_data=(X_test, y_test_one_hot))

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test_one_hot, axis=1)
print_scores(y_test, y_pred)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9622/9622 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9129 - loss: 0.2755 - val_accuracy: 0.9294 - val_loss: 0.2395
Epoch 2/5
9622/9622 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9300 - loss: 0.2325 - val_accuracy: 0.9310 - val_loss: 0.2234
Epoch 3/5
9622/9622 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9327 - loss: 0.2190 - val_accuracy: 0.9306 - val_loss: 0.2214
Epoch 4/5
9622/9622 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9326 - loss: 0.2187 - val_accuracy: 0.9305 - val_loss: 0.2187
Epoch 5/5
9622/9622 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9332 - loss: 0.2180 - val_accuracy: 0.9307 - val_loss: 0.2187
752/752 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     22356
           1       0.54      0.13      0.21      1698

    accuracy                           0.93     24054
   macro avg       0.74      0.56      0.59     24054
weighted avg       0.91      0.93      0.91    

**Chuyển sang Over Sampling**

In [33]:
data['SeriousDlqin2yrs'].value_counts()

,count
SeriousDlqin2yrs,
0,111912
1,8357


In [34]:
df0 = data[data['SeriousDlqin2yrs']==0]
df1 = data[data['SeriousDlqin2yrs']==1]

In [37]:
df_0_new = df0.sample(n=50000, random_state=42)

In [41]:
df = pd.concat([df_0_new, df1], axis=0, ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [44]:
X_train, X_test, y_train, y_test = preprocess_data(df)

In [45]:
sm = SMOTE(k_neighbors=5)
X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

**1.Random Forest**

In [50]:
#Random Forrest
model_RF_new= RandomForestClassifier(n_estimators=200, random_state=42)
model_RF_new.fit(X_train_resample, y_train_resample)

y_pred = model_RF_new.predict(X_test)

print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.92      0.84      0.88      9990
           1       0.38      0.57      0.45      1682

    accuracy                           0.80     11672
   macro avg       0.65      0.71      0.67     11672
weighted avg       0.84      0.80      0.82     11672



**2.XGBOOST**

In [51]:
model_XGB_new = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_XGB_new.fit(X_train_resample, y_train_resample)

y_pred = model_XGB_new.predict(X_test)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.79      0.85      9990
           1       0.33      0.62      0.43      1682

    accuracy                           0.77     11672
   macro avg       0.63      0.71      0.64     11672
weighted avg       0.84      0.77      0.79     11672

